In [13]:
import keras
from keras import layers
import gymnasium as gym
from gymnasium.wrappers.frame_stack import FrameStack
from gymnasium.wrappers.atari_preprocessing import AtariPreprocessing
import numpy as np
import tensorflow as tf
import ale_py

gym.register_envs(ale_py)

# Hyperparameters
seed = 42
gamma = 0.99
epsilon = 1.0
epsilon_min = 0.1
epsilon_max = 1.0
epsilon_interval = (epsilon_max - epsilon_min)
batch_size = 32
max_steps_per_episode = 10000
max_frames = 1e7  # Set a high max_frames to continue running

# Create environment
env = gym.make("SpaceInvaders-v4", render_mode="rgb_array")

# Apply Atari preprocessing with frame_skip=1 (to avoid double frame-skipping)
env = AtariPreprocessing(env, frame_skip=1)

# Apply FrameStack for stacking 4 frames
env = FrameStack(env, 4)

# Video recording settings
trigger = lambda t: t % 1000 == 0
env = gym.wrappers.RecordVideo(env, video_folder="./videos", episode_trigger=trigger, disable_logger=True)

num_actions = 6

def create_q_model():
    return keras.Sequential([
        layers.Lambda(
            lambda tensor: tf.transpose(tensor, [0, 2, 3, 1]),
            output_shape=(84, 84, 4),
            input_shape=(4, 84, 84)
        ),
        layers.Conv2D(32, kernel_size=8, strides=4, activation="relu"),
        layers.Conv2D(64, kernel_size=4, strides=2, activation="relu"),
        layers.Conv2D(64, kernel_size=3, strides=1, activation="relu"),
        layers.Flatten(),
        layers.Dense(512, activation="relu"),
        layers.Dense(num_actions, activation="linear")
    ])

# Initialize models
model = create_q_model()
model_target = create_q_model()

# Optimizer and loss function
optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)
loss_function = keras.losses.Huber()

# Initialize variables
action_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []
episode_reward_history = []
running_reward = 0
episode_count = 0
frame_count = 0

# Hyperparameters for exploration and training
epsilon_random_frames = 50000
epsilon_greedy_frames = 1000000.0
max_memory_length = 1000000
update_after_actions = 4
update_target_network = 10000

# Main training loop
while frame_count < max_frames:
    observation, _ = env.reset()
    state = np.array(observation)
    episode_reward = 0

    for timestep in range(1, max_steps_per_episode):
        frame_count += 1

        # Epsilon-greedy exploration
        if frame_count < epsilon_random_frames or epsilon > np.random.rand(1)[0]:
            action = np.random.choice(num_actions)
        else:
            state_tensor = tf.convert_to_tensor(state)
            state_tensor = tf.expand_dims(state_tensor, 0)  # Add batch dimension
            action_probs = model(state_tensor, training=False)
            action = tf.argmax(action_probs[0]).numpy()

        # Decay epsilon
        epsilon -= epsilon_interval / epsilon_greedy_frames
        epsilon = max(epsilon, epsilon_min)

        # Apply action in environment
        state_next, reward, done, _, _ = env.step(action)
        state_next = np.array(state_next)
        episode_reward += reward

        # Save experience in replay buffer
        action_history.append(action)
        state_history.append(state)
        state_next_history.append(state_next)
        done_history.append(done)
        rewards_history.append(reward)
        state = state_next

        # Update every 4 actions and when enough experiences are stored
        if frame_count % update_after_actions == 0 and len(done_history) > batch_size:
            indices = np.random.choice(range(len(done_history)), size=batch_size)

            state_sample = np.array([state_history[i] for i in indices])
            state_next_sample = np.array([state_next_history[i] for i in indices])
            rewards_sample = [rewards_history[i] for i in indices]
            action_sample = [action_history[i] for i in indices]
            done_sample = tf.convert_to_tensor([float(done_history[i]) for i in indices])

            # Get future rewards using the target model
            future_rewards = model_target.predict(state_next_sample, verbose=0)
            updated_q_values = rewards_sample + gamma * tf.reduce_max(future_rewards, axis=1)

            # Adjust for terminal states
            updated_q_values = updated_q_values * (1 - done_sample) - done_sample

            # One-hot encoding for actions
            masks = tf.one_hot(action_sample, num_actions)

            with tf.GradientTape() as tape:
                q_values = model(state_sample)
                q_action = tf.reduce_sum(q_values * masks, axis=1)
                loss = loss_function(updated_q_values, q_action)

            # Backpropagation and update model weights
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # Update target network periodically
        if frame_count % update_target_network == 0:
            model_target.set_weights(model.get_weights())

            # Print progress and save checkpoint
            print(f"Best score of last 100: {np.max(episode_reward_history)}, "
                  f"Running reward: {running_reward:.2f} at episode {episode_count}, frame count {frame_count}")
            model.save(f"breakout_qmodel_{episode_count}.keras")

        # Manage memory size
        if len(rewards_history) > max_memory_length:
            del rewards_history[:1]
            del state_history[:1]
            del state_next_history[:1]
            del action_history[:1]
            del done_history[:1]

        if done:
            break

    # Update running reward after each episode
    episode_reward_history.append(episode_reward)
    if len(episode_reward_history) > 100:
        del episode_reward_history[:1]
    running_reward = np.mean(episode_reward_history)

    episode_count += 1

    # Check if the task is solved
    if running_reward > 40:
        print(f"Solved at episode {episode_count}, but continuing training...")

    # Save model at regular intervals (every 10 episodes)
    if episode_count % 10 == 0:
        model.save(f"breakout_qmodel_checkpoint_{episode_count}.keras")

# Final model save
model.save("final_breakout_qmodel.keras")
print(f"Training finished after {episode_count} episodes and {frame_count} frames.")


/Users/kevin/Documents/Github/Deep-Learning-Kevin-Fu/venv/lib/python3.12/site-packages/gymnasium/wrappers/monitoring/video_recorder.py:178: UserWarning: WARN: Unable to save last video! Did you call close()?
  logger.warn("Unable to save last video! Did you call close()?")


KeyboardInterrupt: 